In [1]:
import numpy as np
from tabulate import tabulate
from math import pow, sin, cos

Заданные функции

In [2]:
def f(x, y, z):
    return ((-2 * pow(x, 3) * z) - y) / (pow(x, 4))
def g(x, y, z):
    return z
def yf(x):
    return (sin(1) + cos(1))*cos(1/x) + (sin(1) - cos(1)) * sin(1/x)

Метод Эйлера

In [3]:
def euler_method(y0, z0, xl, xr, h):
    """
    Решает задачу Коши для ОДУ 2-го порядка методом Эйлера.
    
    Параметры:
    y0 - начальное значение y.
    z0 - начальное значение z.
    xl - начальная точка интервала интегрирования.
    xr - конечная точка интервала интегрирования.
    h - шаг интегрирования.
    
    Возвращает:
    Списки значений x, y, z, y1, z1 на интервале интегрирования.
    """
    x = [i for i in np.arange(xl, xr + h, h)]
    y = [y0]
    z = [z0]
    z1 = [0]
    y1 = [0]
    
    # Основной цикл метода Эйлера
    for i in range(len(x) - 1):
        # Вычисление промежуточных значений z1 и y1
        z1.append(z[i] + h * f(x[i], y[i], z[i]))
        z.append(z[i] + h * (f(x[i], y[i], z[i]) + f(x[i], y[i], z1[i])) / 2)
        y1.append(y[i] + h * g(x[i], y[i], z[i]))
        y.append(y[i] + h * (g(x[i], y[i], z[i]) + g(x[i], y1[i], z[i])) / 2)
    
    return x, y, z, y1, z1

Метод Рунге-Кутты

In [4]:
def runge_kutty_method(y0, z0, xl, xr, h):
    """
    Решает задачу Коши для ОДУ 2-го порядка методом Рунге-Кутты 4-го порядка.
    
    Параметры:
    y0 - начальное значение y.
    z0 - начальное значение z.
    xl - начальная точка интервала интегрирования.
    xr - конечная точка интервала интегрирования.
    h - шаг интегрирования.
    
    Возвращает:
    Списки значений x, y, z, dy, dz, Ky, Kz на интервале интегрирования.
    """
    x = [i for i in np.arange(xl, xr + h, h)]
    y = [y0]
    z = [z0]
    dz = []
    dy = []
    Kz = []
    Ky = []
    
    # Основной цикл метода Рунге-Кутты
    for i in range(len(x) - 1):
        L1 = h * f(x[i], y[i], z[i])
        K1 = h * g(x[i], y[i], z[i])
        L2 = h * f(x[i] + 0.5 * h, y[i] + 0.5 * K1, z[i] + 0.5 * L1)
        K2 = h * g(x[i] + 0.5 * h, y[i] + 0.5 * K1, z[i] + 0.5 * L1)
        L3 = h * f(x[i] + 0.5 * h, y[i] + 0.5 * K2, z[i] + 0.5 * L2)
        K3 = h * g(x[i] + 0.5 * h, y[i] + 0.5 * K2, z[i] + 0.5 * L2)
        L4 = h * f(x[i] + h, y[i] + K3, z[i] + L3)
        K4 = h * g(x[i] + h, y[i] + K3, z[i] + L3)
        Ky.append([K1, K2, K3, K4])
        Kz.append([L1, L2, L3, L4])
        
        # Вычисление приращений dy и dz по формуле Рунге-Кутты
        dy.append((K1 + 2 * K2 + 2 * K3 + K4) / 6)
        dz.append((L1 + 2 * L2 + 2 * L3 + L4) / 6)
        
        y.append(y[i] + dy[i])
        z.append(z[i] + dz[i])
    
    return x, y, z, dy, dz, Ky, Kz



Метод Адамса

In [5]:
def adams_method(y0, z0, xl, xr, h):
    """
    Решает задачу Коши для ОДУ 2-го порядка методом Адамса 4-го порядка.
    
    Параметры:
    y0 - начальное значение y.
    z0 - начальное значение z.
    xl - начальная точка интервала интегрирования.
    xr - конечная точка интервала интегрирования.
    h - шаг интегрирования.
    
    Возвращает:
    Списки значений x, y, z на интервале интегрирования.
    """
    # Инициализация методом Рунге-Кутты для первых четырех точек
    x, y, z, dy, dz, Ky, Kz = runge_kutty_method(y0, z0, xl, xl + 3 * h, h)
    x = [i for i in np.arange(xl, xr + h, h)]
    
    # Основной цикл метода Адамса
    for i in range(3, len(x) - 1):
        # Вычисление значений z и y методом Адамса-Бэшфорта
        z.append(z[i] + h * (55 * f(x[i], y[i], z[i]) - 59 * f(x[i - 1], y[i - 1], z[i - 1]) + 37 * f(x[i - 2], y[i - 2], z[i - 2]) - 9 * f(x[i - 3], y[i - 3], z[i - 3])) / 24)
        y.append(y[i] + h * (55 * g(x[i], y[i], z[i]) - 59 * g(x[i - 1], y[i - 1], z[i - 1]) + 37 * g(x[i - 2], y[i - 2], z[i - 2]) - 9 * g(x[i - 3], y[i - 3], z[i - 3])) / 24)

    return x, y, z
    


Данные согласно варианту

In [6]:
y0 = 1
z0 = 1
xl = 1
xr = 2
h = 0.1

Результаты для метода Эйлера

In [7]:
x, y, z, y1, z1 = euler_method(y0, z0, xl, xr, h)
table = [["k", "xk", "yk", "~yk", "zk", "~zk", "yf", "errk"]]
table1 = [["k", "xk", "yk"]]
for k in range(len(x)):
    table1.append([k, x[k], y[k]])
    table.append([k, x[k], y[k], y1[k], z[k], z1[k], yf(x[k]), yf(x[k]) - y[k]])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
print(tabulate(table1, headers='firstrow', tablefmt='fancy_grid'))

xe2, ye2, z, y1, z1 = euler_method(y0, z0, xl, xr, 2*h)
xe1, ye1, z, y1, z1 = euler_method(y0, z0, xl, xr, h)

[1]
╒═════╤══════╤═════════╤═════════╤═══════════╤═══════════╤═════════╤════════════╕
│   k │   xk │      yk │     ~yk │        zk │       ~zk │      yf │       errk │
╞═════╪══════╪═════════╪═════════╪═══════════╪═══════════╪═════════╪════════════╡
│   0 │  1   │ 1       │ 0       │ 1         │ 0         │ 1       │  0         │
├─────┼──────┼─────────┼─────────┼───────────┼───────────┼─────────┼────────────┤
│   1 │  1.1 │ 1.1     │ 1.1     │ 0.8       │ 0.7       │ 1.08665 │ -0.0133455 │
├─────┼──────┼─────────┼─────────┼───────────┼───────────┼─────────┼────────────┤
│   2 │  1.2 │ 1.18    │ 1.18    │ 0.588505  │ 0.579414  │ 1.15204 │ -0.0279606 │
├─────┼──────┼─────────┼─────────┼───────────┼───────────┼─────────┼────────────┤
│   3 │  1.3 │ 1.23885 │ 1.23885 │ 0.434272  │ 0.433515  │ 1.20222 │ -0.0366333 │
├─────┼──────┼─────────┼─────────┼───────────┼───────────┼─────────┼────────────┤
│   4 │  1.4 │ 1.28228 │ 1.28228 │ 0.324144  │ 0.324086  │ 1.2413  │ -0.0409743 │
├─────┼─────

Результаты для метода Рунге-Кутта

In [8]:
x,y,z,dy,dz,Ky,Kz = runge_kutty_method(y0, z0, xl, xr, h)
table = [["k", "xk", "yk", "dyk", "zk", "dzk", "yf", "errk"]]
tablek = [["k", "K1", "K2", "K3", "K4", "L1", "L2", "L3", "L4"]]
table1 = [["k", "xk", "yk"]]
for k in range(len(x)):
    if k != len(x) - 1:
        tablek.append([k, Ky[k][0], Ky[k][1], Ky[k][2], Ky[k][3], Kz[k][0], Kz[k][1], Kz[k][2], Kz[k][3]])
    table1.append([k, x[k], y[k]])
    table.append([k, x[k], y[k], y1[k], z[k], z1[k], yf(x[k]), yf(x[k]) - y[k]])
print(tabulate(tablek, headers='firstrow', tablefmt='fancy_grid'))
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
print(tabulate(table1, headers='firstrow', tablefmt='fancy_grid'))

xrk2,yrk2,z,dy,dz,Ky,Kz = runge_kutty_method(y0, z0, xl, xr, 2*h)
xrk1,yrk1,z,dy,dz,Ky,Kz = runge_kutty_method(y0, z0, xl, xr, h)

╒═════╤═══════════╤═══════════╤═══════════╤════════════╤════════════╤════════════╤════════════╤════════════╕
│   k │        K1 │        K2 │        K3 │         K4 │         L1 │         L2 │         L3 │         L4 │
╞═════╪═══════════╪═══════════╪═══════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│   0 │ 0.1       │ 0.085     │ 0.0875856 │ 0.0747404  │ -0.3       │ -0.248289  │ -0.252596  │ -0.210175  │
├─────┼───────────┼───────────┼───────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│   1 │ 0.0748009 │ 0.0642898 │ 0.0659971 │ 0.0569264  │ -0.210221  │ -0.176076  │ -0.178745  │ -0.150464  │
├─────┼───────────┼───────────┼───────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│   2 │ 0.0569621 │ 0.0494374 │ 0.0505894 │ 0.0440478  │ -0.150494  │ -0.127454  │ -0.129143  │ -0.109873  │
├─────┼───────────┼───────────┼───────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│   3 │ 0.0440694 │

Результаты для метода Адамса

In [9]:
x,y,z = adams_method(y0, z0, xl, xr, h)
table = [["k", "xk", "yk", "zk", "yf", "errk"]]
table1 = [["k", "xk", "yk"]]
for k in range(len(x)):
    table1.append([k, x[k], y[k]])
    table.append([k, x[k], y[k], z[k], yf(x[k]), yf(x[k]) - y[k]])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
print(tabulate(table1, headers='firstrow', tablefmt='fancy_grid'))

xa2,ya2,za2 = adams_method(y0, z0, xl, xr, 2*h)
xa1,ya1,za1 = adams_method(y0, z0, xl, xr, h)

╒═════╤══════╤═════════╤══════════╤═════════╤═════════════╕
│   k │   xk │      yk │       zk │      yf │        errk │
╞═════╪══════╪═════════╪══════════╪═════════╪═════════════╡
│   0 │  1   │ 1       │ 1        │ 1       │ 0           │
├─────┼──────┼─────────┼──────────┼─────────┼─────────────┤
│   1 │  1.1 │ 1.08665 │ 0.748009 │ 1.08665 │ 2.6192e-06  │
├─────┼──────┼─────────┼──────────┼─────────┼─────────────┤
│   2 │  1.2 │ 1.15204 │ 0.569621 │ 1.15204 │ 3.91337e-06 │
├─────┼──────┼─────────┼──────────┼─────────┼─────────────┤
│   3 │  1.3 │ 1.20221 │ 0.440694 │ 1.20222 │ 4.46289e-06 │
├─────┼──────┼─────────┼──────────┼─────────┼─────────────┤
│   4 │  1.4 │ 1.2413  │ 0.345729 │ 1.2413  │ 4.60265e-06 │
├─────┼──────┼─────────┼──────────┼─────────┼─────────────┤
│   5 │  1.5 │ 1.24099 │ 0.347232 │ 1.27215 │ 0.0311603   │
├─────┼──────┼─────────┼──────────┼─────────┼─────────────┤
│   6 │  1.6 │ 1.27196 │ 0.275467 │ 1.29678 │ 0.024823    │
├─────┼──────┼─────────┼──────────┼─────

Погрешность Рунге-Ромберга

In [10]:
def runge_rom(y1, y2, r, p):
    k = -10
    for i in range(min(len(y1), len(y2))):
        k = max(k, abs((y1[i] - y2[i]) / (r**p - 1))) 
    return k

table = [["Эйлера", "Рунге-Кутты", "Адамса"]]
table.append([runge_rom(ye1, ye2, 2, 2), runge_rom(yrk1, yrk2, 2, 4), runge_rom(ya1, ya2, 2, 4)])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒═══════════╤═══════════════╤════════════╕
│    Эйлера │   Рунге-Кутты │     Адамса │
╞═══════════╪═══════════════╪════════════╡
│ 0.0482227 │    0.00629909 │ 0.00793684 │
╘═══════════╧═══════════════╧════════════╛
